In [ ]:
import matplotlib.pyplot as plt
from utilities import *
from sklearn.datasets import make_blobs, make_circles
from sklearn.metrics import accuracy_score, log_loss
from scipy.signal import convolve2d
from tqdm import tqdm
# On specifie a matplotlib que l'on est en dark mode
plt.style.use('dark_background')

# Fonctions du réseau de neurones
## Initialisation des paramètres

In [ ]:
# Fonction d'intialisation des parametres
# entree : dimension (liste) : liste des dimensions des couches
# sortie : parametres (dictionnaire) : dictionnaire des parametres initialises
def initialisation(dimensions):
    
    parametres = {}
    C = len(dimensions) # nombre de couches
    
    for c in range (1,C): # pour chaque couche (sauf la premiere)
        parametres['W' + str(c)] = np.random.randn(dimensions[c], dimensions[c-1]) # initialisation aleatoire des poids
        parametres['b' + str(c)] = np.random.randn(dimensions[c], 1) # initialisation des biais a zero

    return parametres

Test de la fonction d'initialisation

In [ ]:
# Test de la fonction d'initialisation
parametres = initialisation([2, 32, 32, 1]) # Initialisation d'un RN avec :
# 2 neurones en entree
# 32 neurones sur chaque couche cachée
# 1 neurone de sortie
for k, v in parametres.items():
    print(k, v.shape)

## Propagation vers l'avant

In [ ]:
# Fonction de propagation avant
# entree :  - X (np.array) : donnees d'entree
#           - parametres (dictionnaire) : dictionnaire des parametres
# sortie : activations (dictionnaire) : dictionnaire des activations

def forward_propagation(X, parametres):
    activations = {'A0': X} # initialisation : la premiere activation est l'entree
    
    C = len(parametres) // 2 # nombre de couches
    
    for c in range(1, C + 1): # pour chaque couche
        Z = parametres['W' + str(c)].dot(activations['A' + str(c - 1)]) + parametres['b' + str(c)] # calcul de Z
        activations['A' + str(c)] = 1 / (1 + np.exp(-Z)) # calcul de A
    return activations

Test de la fonction de propagation avant

In [ ]:
# Test de la fonction de propagation avant
X = np.array([[1, 2], [3, 4]]) # Donnees d'entree
y = np.array([[0, 1]]) # Donnees de sortie
activations = forward_propagation(X, parametres) # Propagation avant : avec en entree du premier neurone 1 et 3, et le deuxieme neurone 2 et 4.
for k, v in activations.items():
    print(k, v.shape)

## Retropropagation

In [ ]:
def back_propagation(y, parametres, activations):

  m = y.shape[1]
  C = len(parametres) // 2

  dZ = activations['A' + str(C)] - y
  gradients = {}

  for c in reversed(range(1, C + 1)):
    gradients['dW' + str(c)] = 1/m * np.dot(dZ, activations['A' + str(c - 1)].T)
    gradients['db' + str(c)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    if c > 1:
      dZ = np.dot(parametres['W' + str(c)].T, dZ) * activations['A' + str(c - 1)] * (1 - activations['A' + str(c - 1)])

  return gradients

Test de la fonction de retropropagation

In [ ]:
# Test de la fonction de retropropagation
gradients = back_propagation(y, parametres, activations)
for k, v in gradients.items():
    print(k, v.shape)

## Mise à jour des paramètres (descente de gradient)

In [ ]:
def update(gradients, parametres, learning_rate):

    C = len(parametres) // 2

    for c in range(1, C + 1):
        parametres['W' + str(c)] = parametres['W' + str(c)] - learning_rate * gradients['dW' + str(c)]
        parametres['b' + str(c)] = parametres['b' + str(c)] - learning_rate * gradients['db' + str(c)]

    return parametres

## Prédiction de données

In [ ]:
def predict(X, parametres):
  activations = forward_propagation(X, parametres)
  C = len(parametres) // 2
  Af = activations['A' + str(C)]
  return Af>= 0.5

## Réseau de neurones profond (fonction globale)
Dans cette fonction, on va entrainer un réseau de neurones profond sur un jeu de données donné.
On commence par initialiser les paramètres du réseau, puis on effectue une descente de gradient pour minimiser la fonction de coût (log_loss).
On affiche ensuite la courbe d'apprentissage (log_loss et accuracy) et on retourne les paramètres du réseau.

Les paramètres de la fonction sont :
- X : les données d'entrée
- y : les données de sortie
- hidden_layers : un tuple contenant le nombre de neurones par couche cachée
- learning_rate : le taux d'apprentissage (Pour doser ce paramètre, il faut tester plusieurs valeurs, en général on commence par 0.1 puis on ajuste. Si le taux d'apprentissage est trop grand, l'algorithme peut diverger, si il est trop petit, l'algorithme peut mettre trop de temps à converger)
- n_iter : le nombre d'itérations de la descente de gradient (plus il y a d'itérations, plus le modèle aura de chance de converger, mais plus le temps de calcul sera long)

La fonction retourne un tuple contenant :
- training_history : un tableau numpy contenant les valeurs de log_loss et d'accuracy à chaque itération
- parametres : les paramètres du réseau de neurones entrainé

In [ ]:
def deep_neural_network(X, y, hidden_layers = (16, 16, 16), learning_rate = 0.001, n_iter = 3000):
    print("Learning rate : ", learning_rate, "Number of iterations : ", n_iter)
    
    # initialisation parametres
    dimensions = list(hidden_layers) # liste des dimensions des couches
    dimensions.insert(0, X.shape[0]) # ajout de la dimension de l'entree
    dimensions.append(y.shape[0]) # ajout de la dimension de la sortie
    np.random.seed(1) # fixer la seed pour reproduire les resultats
    parametres = initialisation(dimensions) # initialisation des parametres
    
    # affiche un shemas de la structure du reseau
    print("Structure du reseau :")
    for i in range(0, len(dimensions)):
        print("Couche ", i, " : ", dimensions[i], " neurones")
    print()
    
    # tableau numpy contenant les futures accuracy et log_loss
    training_history = np.zeros((int(n_iter), 2))

    C = len(parametres) // 2

    # gradient descent
    for i in tqdm(range(n_iter)):

        activations = forward_propagation(X, parametres)
        gradients = back_propagation(y, parametres, activations)
        parametres = update(gradients, parametres, learning_rate)
        Af = activations['A' + str(C)]

        # calcul du log_loss et de l'accuracy
        training_history[i, 0] = (log_loss(y.flatten(), Af.flatten()))
        y_pred = predict(X, parametres)
        training_history[i, 1] = (accuracy_score(y.flatten(), y_pred.flatten()))

    # Plot courbe d'apprentissage
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(training_history[:, 0], label='train loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(training_history[:, 1], label='train acc')
    plt.legend()
    plt.show()

    return training_history, parametres

# Initialisation d'un dataset 2D
## On va commencer par initialiser un dataset 2D pour tester notre réseau de neurones.
On va créer un dataset 2D avec 2 classes, puis on va entrainer notre réseau de neurones sur ce dataset. On va ensuite afficher la ligne de décision du réseau de neurones.

Pour cela, on va :
 - Utiliser des fonctions de sklearn pour générer des datasets 2D.
  - make_blobs : génère des points appartenant à 2 classes, les points de chaque classe sont regroupés.
  - make_circles : génère des points appartenant à 2 classes, les points de chaque classe sont disposés en cercle.
  - On va aussi créer un dataset 2D en générant des points aléatoires et en attribuant une classe en fonction de la distance de chaque point à l'origine.
  - On va aussi créer un dataset 2D en générant des points aléatoires et en attribuant une classe en fonction de la position du point par rapport à une fonction cosinus.   
 - On va ensuite afficher les points du dataset avec la fonction scatter de matplotlib.pyplot, en coloriant les points en fonction de leur classe.
 - On va ensuite afficher la ligne de décision du réseau de neurones en utilisant la fonction contourf de matplotlib.pyplot, qui permet de tracer des lignes de niveau.

### Note : 
 - Les fonctions de génération de dataset génèrent aussi un dataset de test pour évaluer le réseau de neurones dans des conditions similaires à l'entrainement.
 - Les fonctions de génération de dataset retournent les données sous forme de matrices numpy, avec les points en colonnes et les classes en lignes.
  - X est une matrice de dimension (2, n) où n est le nombre de points.
  - y est une matrice de dimension (1, n) où n est le nombre de points.

In [ ]:
# Dataset 2D de sklearn
def dataset1_0():
    X, y = make_blobs(n_samples=100, n_features=2, centers=2, random_state=0)
    y = y.reshape((1, y.shape[0]))
    X = X.T
    print(X.shape, y.shape)
    return X,y

def dataset1_1():
    X, y = make_circles(n_samples=100, noise=0.1, factor=0.3, random_state=0)
    X = X.T
    y = y.reshape((1, y.shape[0]))
    print(X.shape, y.shape)
    return X,y

# Dataset 2D
def dataset1_2():
    X_train = np.random.randn(1000, 2)
    X_test = np.random.randn(100, 2)
    y_train = np.zeros((1000, 1))
    y_test = np.zeros((100, 1))
    
    for i in range(1000):
        # si la distance de X[i] à l'origine est inférieure à 1 alors le point est de la classe 1 sinon de la classe 0
        if np.linalg.norm(X_train[i]) < 1:
            y_train[i] = 1
        else:
            y_train[i] = 0
        if i < 100:
            if np.linalg.norm(X_test[i]) < 1:
                y_test[i] = 1
            else:
                y_test[i] = 0
        

    X_train = X_train.T
    X_test = X_test.T
    y_train = y_train.T
    y_test = y_test.T
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    return X_train,y_train,X_test,y_test

def dataset1_3(x1, x2, y1, y2):
    X_train = np.zeros((1500, 2))
    X_test = np.zeros((100, 2))
    y_train = np.zeros((1500, 1))
    y_test = np.zeros((100, 1))
    
    
    for i in range(1500):
        random_x = np.random.uniform(x1, x2)
        random_y = np.random.uniform(y1, y2)
        X_train[i] = np.array([random_x, random_y])
        # si le point est au dessus de la fonction cosinus alors il est de la classe 1 sinon de la classe 0
    
        if random_y > np.cos(random_x):
            y_train[i] = 1
        else:
            y_train[i] = 0
        if i < 100:
            random_x_test = np.random.uniform(x1, x2)
            random_y_test = np.random.uniform(y1, y2)
            X_test[i] = np.array([random_x_test, random_y_test])
            if random_y_test > np.cos(random_x_test):
                y_test[i] = 1
            else:
                y_test[i] = 0
    
    X_train = X_train.T
    X_test = X_test.T
    y_train = y_train.T
    y_test = y_test.T
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    return X_train,y_train,X_test,y_test

def affichage_train(X,y):
    # affiche les points
    plt.scatter(X[0, :], X[1, :], c=y.flatten(), cmap=plt.cm.coolwarm)
    plt.show()
    
def affichage_data(X_train, y_train, X_test, y_test):
    print("Train set :")
    print(X_train.shape)
    print(y_train.shape)
    print(np.unique(y_train, return_counts=True))
    print()
    print("Test set :")
    print(X_test.shape)
    print(y_test.shape)
    print(np.unique(y_test, return_counts=True))

# Initialisation d'un dataset 2D
## On va commencer par initialiser un dataset 2D pour tester notre réseau de neurones.

In [ ]:
# X_train,y_train = dataset1_0()
# X_train,y_train = dataset1_1()
# X_train, y_train, X_test, y_test = dataset1_2()
X_train, y_train, X_test, y_test = dataset1_3(0, 10, -1.5, 1.5)

# Affichage du dataset
## On va afficher les points du dataset avec la fonction scatter de matplotlib.pyplot, en coloriant les points en fonction de leur classe.

In [ ]:
affichage_train(X_train,y_train.T)

# Entrainement du réseau de neurones

## Entrainement sur le Dataset

Nous allons maintenant entraîner notre réseau de neurones sur ce jeu de données.

Pour cela, nous utiliserons la fonction `deep_neural_network` en fournissant les données d'entraînement. Nous spécifierons le nombre de neurones par couche cachée, le taux d'apprentissage et le nombre d'itérations de la descente de gradient.

La fonction renverra les paramètres du réseau de neurones entraîné, ainsi qu'un tableau numpy contenant les valeurs de la perte logarithmique (log_loss) et de la précision (accuracy) à chaque itération.

### Précision (Accuracy) :

La précision est définie comme le nombre de prédictions correctes divisé par le nombre total de prédictions.

### Perte Logarithmique (Log Loss) :

La perte logarithmique (ou log_loss) est une mesure de l'erreur de prédiction. Elle est définie par la formule suivante :

\[ \text{Log Loss} = -\frac{1}{n} \sum_{i=1}^{n} y_i \log(y_{\text{pred}_i}) + (1 - y_i) \log(1 - y_{\text{pred}_i}) \] 

Où :
- \( y \) est le vecteur des vraies valeurs.
- \( y_{\text{pred}} \) est le vecteur des valeurs prédites par le réseau de neurones.
- \( n \) est le nombre total d'exemples dans le jeu de données.

Plus la perte logarithmique est proche de 0, meilleure est la performance du réseau de neurones dans la prédiction des données.
### Si jamais la formule n'est pas affichée correctement : https://www.codecogs.com/latex/eqneditor.php

In [ ]:
# Entrainement du reseau de neurones
history, params = deep_neural_network(X_train, y_train, hidden_layers = (32, 32, 32, 32), learning_rate = 0.1, n_iter = 10000)

# Prédiction des nouvelles données
On va maintenant prédire les classes des points du dataset de test en utilisant la fonction `predict`.
On va ensuite afficher la ligne de décision du réseau de neurones en utilisant la fonction `affichage_decision`.
- Le premier plot affiche les points de train avec la ligne de decision
- Le deuxieme plot affiche les points de test avec la ligne de decision (si la prédiction est juste le point est simbolisé en une croix sinon en un rond)

In [ ]:
# Prédiction des nouvelles données
predictions = predict(X_test, params)

# Affiche la ligne de decision dans 2 plots :
# - le premier plot affiche les points de train avec la ligne de decision
# - le deuxieme plot affiche les points de test avec la ligne de decision (si la prédiction est juste le point est simbolisé en une croix sinon en un rond)
def affichage_decision(X_train, y_train, X_test, predictions, y_test, params):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Plot pour les données d'entraînement
    axes[0].scatter(X_train[0, :], X_train[1, :], c=y_train.flatten(), cmap=plt.cm.coolwarm)
    axes[0].set_title('Entraînement')
    plt.legend()

    # Affichage de la ligne de décision pour les données d'entraînement
    x = np.linspace(np.min(X_train[0, :]), np.max(X_train[0, :]), 100)
    y = np.linspace(np.min(X_train[1, :]), np.max(X_train[1, :]), 100)
    xx, yy = np.meshgrid(x, y)
    Z = np.zeros(xx.shape)
    for i in range(xx.shape[0]):
        for j in range(xx.shape[1]):
            Z[i, j] = predict(np.array([[xx[i, j]], [yy[i, j]]]), params)[0, 0]
    axes[0].contourf(xx, yy, Z, alpha=0.5)

    # Plot pour les données de test
    axes[1].scatter(X_test[0, :], X_test[1, :], c=y_test.flatten(), cmap=plt.cm.coolwarm, label='Erreures de prédiction', edgecolors='k')
    axes[1].set_title('Test')
    plt.legend()

    # Affichage de la ligne de décision pour les données de test
    x = np.linspace(np.min(X_test[0, :]), np.max(X_test[0, :]), 100)
    y = np.linspace(np.min(X_test[1, :]), np.max(X_test[1, :]), 100)
    xx, yy = np.meshgrid(x, y)
    Z = np.zeros(xx.shape)
    for i in range(xx.shape[0]):
        for j in range(xx.shape[1]):
            Z[i, j] = predict(np.array([[xx[i, j]], [yy[i, j]]]), params)
    axes[1].contourf(xx, yy, Z, alpha=0.5)

    # Ajout des marqueurs pour les prédictions correctes et incorrectes
    for i in range(X_test.shape[1]):
        if predictions[0, i] == y_test[0, i]:
            axes[1].scatter(X_test[0, i], X_test[1, i], marker='x', c='green')
        else:
            axes[1].scatter(X_test[0, i], X_test[1, i], marker='o', c='red')

    plt.show()

affichage_decision(X_train, y_train, X_test, predictions, y_test, params)

# Initialisation d'un dataset (images)
## On va maintenant initialiser un dataset d'images pour tester notre réseau de neurones.
On utilise un dataset de 1200 images de 64x64 pixels en noir et blanc de chats et de chiens. Les images sont divisées en deux classes : les chats et les chiens. On prend 1000 images pour l'entrainement et 200 images pour le test.

In [ ]:
def dataset3():
    return load_data()

In [ ]:
X_train, y_train, X_test, y_test = dataset3()
def affiche_image (X,y): # affiche les 25 premières images avec leur label
    plt.figure(figsize=(10, 10))
    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.imshow(X[i], cmap='gray')
        plt.title(f"Label: {y[i]}")
        plt.axis('off')
    plt.show()
    
def affiche_image_erreur (X,y, predictions):
    plt.figure(figsize=(50, 25))
    predictions_named = np.zeros(200, dtype=object)
    for i in range(200):
        if predictions[i] == 0:
            predictions_named[i] = "Chat"
        else:
            predictions_named[i] = "Chien"
        plt.subplot(10, 20, i + 1)
        if y[i] == predictions[i]:
            plt.imshow(X[i], cmap='gray')
        else:
            plt.imshow(X[i], cmap='Reds')
        plt.title(f"Label: {predictions_named.T[i]}")
        plt.axis('off')
    plt.show()
    
    
# on affiche les 200 premières images avec leur label, en rouge si la prédiction est fausse
# on affiche en premier les images mal prédites (en parcoutant les 200 premières images, 
# on adapte l'affichage pour pas qu'il y ait trop de trous) puis les images bien prédites
def affiche_image_erreur_sorted (X,y,predictions):
    plt.figure(figsize=(50, 25))
    predictions_named = np.zeros(X.shape[0], dtype=object)
    nb_erreur = 0
    for i in range(200):
        if predictions[i] == 0: # 0 = chat
            predictions_named[i] = "Chat"
        else:
            predictions_named[i] = "Chien"
        if y[i] != predictions[i]: # si la prédiction est fausse
            plt.subplot(10, 20, 200-nb_erreur)
            nb_erreur += 1
            plt.imshow(X[i], cmap='Reds')
        else:
            plt.subplot(10, 20, i+1-nb_erreur)
            plt.imshow(X[i], cmap='gray')
        plt.title(f"Label: {predictions_named.T[i]}")
        plt.axis('off')
    print("Nombre d'erreurs : ",nb_erreur,"/",200,"\nAccuracy de ",100 - nb_erreur/200*100,"%")
    plt.show()

In [ ]:
affichage_data(X_train, y_train, X_test, y_test)
affiche_image(X_train,y_train)

# Modification des images
## On va maintenant modifier les images pour les rendre plus facilement traitables par notre réseau de neurones.
 - On commence par normaliser les images, c'est-à-dire diviser les valeurs des pixels par 255 pour les ramener entre 0 et 1.
 - On aplatit les images, c'est-à-dire transformer les matrices 2D des images en vecteurs 1D.
 - On applique un filtre de Sobel sur les images pour mettre en évidence les contours des objets dans les images.

In [ ]:
def filtreSobelProf(img):
    # Ici le filtrage est 2D.
    # Fitre de Sobel 2D en lignes
    Gx = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
    # Filgre de Sobel 2D en colonnnes
    Gy = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
 
    # On crée une image 2D pour le filtre de Sobel en lignes
    sobelX = convolve2d(img, Gx, 'same')
 
    # On crée une image 2D pour le filtre de Sobel en colonnes
    sobelY = convolve2d(img, Gy, 'same')
 
    # On calcule la norme de ces 2 images
    sobelN = np.zeros((img.shape[0], img.shape[1]), np.dtype(float))
    for li in range(img.shape[0]):
        for col in range(img.shape[1]):
            sobelN[li, col] = np.sqrt(sobelX[li, col]*sobelX[li, col] + sobelY[li, col]*sobelY[li, col])
 
    return sobelN


# Normalize function
def normalize_data(data):
    return data / 255.0

# Flatten function
def flatten_data(data):
    return data.reshape(data.shape[0], -1)

# Application sur les images

X_train_Sobel = np.zeros((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test_Sobel = np.zeros((X_test.shape[0], X_test.shape[1], X_test.shape[2]))
# Sobelize the images
for i in range(X_train.shape[0]):
    X_train_Sobel[i] = filtreSobelProf(X_train[i])
for i in range(X_test.shape[0]):
    X_test_Sobel[i] = filtreSobelProf(X_test[i])

# Normalize the train set and the test set
X_train_normalized = normalize_data(X_train_Sobel)
X_test_normalized = normalize_data(X_test_Sobel)

# Flatten the images in the train set and the test set
X_train_flattened = flatten_data(X_train_normalized)
X_test_flattened = flatten_data(X_test_normalized)

y_train_flattened = y_train.flatten()
y_test_flattened = y_test.flatten()

# reshape y_train and y_test
y_train = y_train.T
y_test = y_test.T

# Transpose the data
X_train_flattened = X_train_flattened.T
X_test_flattened = X_test_flattened.T

# Print the shapes after normalization and flattening
print("Train set after normalization and flattening:")
print("X_train shape:", X_train_flattened.shape)
print("y_train shape:", y_train.shape)
print()
print("Test set after normalization and flattening:")
print("X_test shape:", X_test_flattened.shape)
print("y_test shape:", y_test.shape)

# Entrainement du réseau de neurones sur les images
## On va maintenant entraîner notre réseau de neurones sur les images normalisées et aplaties.

In [ ]:
# Train the model on the train_set
training_history, trained_parameters = deep_neural_network(X_train_flattened, y_train, hidden_layers=(512,256,128,64,10), learning_rate=0.1, n_iter=500)

# Evaluate the model on the test_set
test_loss = training_history[-1, 0]  # Final training loss can be used as test loss
print("Test set loss:", test_loss)

# Prédiction des nouvelles données
On va maintenant prédire les classes des images du dataset de test en utilisant la fonction `predict`.
On va afficher l'accuracy du modèle et le nombre d'erreurs sur le dataset de test.
On va ensuite afficher les images du dataset de test avec leur label, en rouge si la prédiction est fausse.

In [ ]:
def affiche_resultat(X, y, predictions):
    # affiche l'accuarcy
    print("Accuracy : ", accuracy_score(y, predictions))
    affiche_image_erreur_sorted(X, y, predictions)

# Predict the test set
predictions = predict(X_test_flattened, trained_parameters)
affiche_resultat(X_test, y_test_flattened, predictions.flatten())

# Test avec un réseau de neurones convolutif

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_train_normalized, y_train_flattened, test_size=0.2, random_state=42)

# One hot encode the labels
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test_flattened)

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=categorical_crossentropy, metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train.reshape(-1, 64, 64, 1), y_train, validation_data=(X_val.reshape(-1, 64, 64, 1), y_val), epochs=100, callbacks=[early_stopping])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized.reshape(-1, 64, 64, 1), y_test)
print("Test set loss:", test_loss)
print("Test set accuracy:", test_accuracy)

# Prédiction des nouvelles données
On va maintenant prédire les classes des images du dataset de test en utilisant la fonction `predict`.
On va afficher l'accuracy du modèle et le nombre d'erreurs sur le dataset de test.
On va ensuite afficher les images du dataset de test avec leur label, en rouge si la prédiction est fausse.

In [ ]:
# Predict the test set
predictions = model.predict(X_test_normalized.reshape(-1, 64, 64, 1))
predictions = np.argmax(predictions, axis=1)
affiche_resultat(X_test, y_test_flattened, predictions)

# Conclusion

Dans cette étude, nous avons entraîné deux types de réseaux neuronaux : un réseau de neurones profond sur un jeu de données 2D et un réseau de neurones convolutif sur un jeu de données d'images de chats et de chiens. 

Nous avons réalisé plusieurs étapes :
- Entraînement et affichage de la courbe d'apprentissage du réseau de neurones profond sur le jeu de données 2D, ainsi que l'affichage de la ligne de décision.
- Affichage des images du jeu de données d'images de chats et de chiens avec leurs étiquettes, en colorant en rouge les prédictions incorrectes.
- Entraînement d'un réseau de neurones convolutif sur le jeu de données d'images de chats et de chiens.
- Affichage de l'accuracy du modèle ainsi que du nombre d'erreurs sur le jeu de données de test, et des images avec leurs étiquettes, en marquant en rouge les prédictions incorrectes.
- Comparaison des performances des deux modèles.

Il est clair que le réseau de neurones convolutif surpasse le réseau de neurones profond dans la classification d'images de chats et de chiens. Cette supériorité s'explique par la capacité des réseaux de neurones convolutifs à prendre en compte la structure spatiale des images. Ces réseaux utilisent des filtres pour extraire des caractéristiques des images, ce qui les rend plus performants dans ce type de tâches.

En conclusion, les réseaux de neurones convolutifs sont plus adaptés pour le traitement des images que les réseaux de neurones profonds classiques.